# **Conjunto de dados 1**

Perguntas e respostas geradas pela LLM, fornecendo como base um conjunto de
artigos.

Essas perguntas serão avaliadas por outras LLMs em três aspectos:

- Qualidade da pergunta: Se a pergunta realmente faz sentido com o artigo usado na sua criação
- Fidelidade da resposta: Se a resposta está correta de acordo com o artigo usado
- Responsibilidade se a pergunta é de fato respondida pela resposta e pelo artigo.

Extras:
 - Avaliações de acordo com scores
 - Uso de hipoteses ou histórias hipoteticas

# ENV SETUP

In [ ]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.1 MB/s eta 0:00:00


# Imports e Defines

In [ ]:
import json
import random
import numpy as np
import pandas as pd
from getpass import getpass
from tqdm import tqdm

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Set the random seed
random.seed(42)

In [ ]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


# LOAD DATA FILES

In [ ]:
with open("codigo_civil.json", "r") as f:
    cod_civil = json.load(f)
with open("/content/QAsites_FAMILIA_SUCESSOES.json", "r") as f:
    examples = json.load(f)

In [ ]:
examples_df = pd.DataFrame(examples)

# Exclude some lines based on manual analyses
examples_df = examples_df.drop(5) # repetido com index 45
examples_df = examples_df.drop(32) # repetido com index 23
examples_df = examples_df.drop(33) # repetido com index 24, similar ao 26 e 27
examples_df = examples_df.drop(35) # repetido com index 26, similar ao 24 e 27
examples_df = examples_df.drop(36) # repetido com index 27, similar ao 24 e 26
examples_df = examples_df.drop(25) # repetido com index 34
examples_df = examples_df.drop(28) # repetido com index 38
examples_df = examples_df.drop([14, 22, 31, 44]) # sem resposta no Cod. Civil
examples_df = examples_df.drop([17, 21]) # resposta no Cod. Proc. Civil
examples_df = examples_df.drop([19, 20]) # Lei de Introdução às normas do Direito Brasileiro.
examples_df = examples_df.drop([43]) # resposta no EJA.
examples_df = examples_df.reset_index()
len(examples_df)

46

In [ ]:
faq_qa = examples_df[~examples_df['name'].str.contains('granconcursos')].reset_index()
concurso_qa = examples_df[examples_df['name'].str.contains('granconcursos')].reset_index()

len(faq_qa), len(concurso_qa)

(30, 16)

In [ ]:
def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

result = collect_artigo_and_header(cod_civil['parte especial']['LIVRO IV'])
iv_context = ''.join(result['contexts'])
iv_articles = result['articles']

result = collect_artigo_and_header(cod_civil['parte especial']['LIVRO V'])
v_context = ''.join(result['contexts'])
v_articles = result['articles']

# Load LLMs

In [ ]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.019259659, 'prompt_time': 0.005645186, 'queue_time': None, 'total_time': 0.024904845}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-68f2f126-6585-4ceb-a5cd-7bd4fa863a3c-0')

# LLM 1: Gerador de Perguntas e Respostas

In [ ]:
class QA_JSON(BaseModel):
    pergunta: str = Field(description="questão cuja resposta é encontrada no artigo")
    resposta: str = Field(description="resposta curta para a questão")
    justificativa: str = Field(description="razão pelo qual o artigo responde a questão")
    artigos: str = Field(description="numero dos artigos que respondem a questão")
    contexto: str = Field(description="trecho do artigo que responde a questão")

output_parser = JsonOutputParser(pydantic_object=QA_JSON)

In [ ]:
prompt = """
Você é um estudante dedicado de direito do Código Civil Brasileiro. Em especial dos livros IV e V: Direito da Família e Sucessões.
Como trabalho de conclusão de curso você deseja montar um FAQ sobre esses tópicos para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil Brasileiro.
Sua tarefa então é dado um conjunto de artigos elaborar perguntas e responde-las baseando suas respostas em trechos desses artigos.
Para te ajudar irei fornecer 1 exemplo de pergunta e resposta de FAQ para que você se inspire no estilo:
Exemplo FAQ:
{exemplo_pergunta}
{exemplo_resposta}
Não se esqueça de dar um contexto para as suas perguntas.

Agora é sua vez de montar o FAQ.
Utilize esses artigos, esforce-se para criar perguntas e respostas que se utilizam simultaneamente de mais de um deles e usando artigos de livros diferentes.

Artigos do livro IV: {iv_articles}
Artigos do livro V: {v_articles}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["exemplo_pergunta", "exemplo_resposta", "iv_articles", "v_articles"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def text_sample():
    idx = random.randint(0, len(faq_qa)-1)
    x = faq_qa.iloc[idx]
    x = x['pergunta']
    return x, idx

for _ in range(10):
    print(text_sample())

('7. Me casei no exterior, então também sou casada no Brasil?', 15)
('7. Em caso de não haver herdeiros, quem fica com a herança?', 7)
('1) É obrigatório pagar pensão alimentícia?', 25)
('7. Me casei no exterior, então também sou casada no Brasil?', 15)
('1) É obrigatório pagar pensão alimentícia?', 25)
('4. O cônjuge tem direito à pensão após o divórcio?', 13)
('Há alguma exceção onde um enteado possa receber a herança?', 6)
('4. Posso dispor da minha herança em vida?', 3)
('4. Posso dispor da minha herança em vida?', 3)
('19. Como funciona uma audiência de pensão alimentícia?', 21)


In [ ]:
def call_qa_gen_agent(iv_articles, v_articles, examples):
    if type(examples) == pd.core.frame.DataFrame:
        exemplo = examples.iloc[random.randint(0, len(examples)-1)]
        pergunta = exemplo['pergunta']
        resposta = exemplo['resposta']
    else:
        exemplo = random.choice(examples)
        pergunta = exemplo['pergunta']
        resposta = exemplo['resposta']

    iv_article = random.choice(iv_articles)
    v_article = random.choice(v_articles)
    output = chain.invoke({"exemplo_pergunta": pergunta,
                           "exemplo_resposta": resposta,
                           "iv_articles": iv_article,
                           "v_articles": v_article})
    if type(output) == dict:
        output = list(output)

    for out_dict in output:
        out_dict['ref_pergunta'] = pergunta
        out_dict['ref_resposta'] = resposta
        out_dict['ref_artigos'] = f"{iv_article}, {v_article}"

    return output

# test
call_qa_gen_agent(iv_articles, v_articles, examples)

[{'pergunta': 'O que é o regime de participação final nos aqüestos?',
  'resposta': 'É o regime em que cada cônjuge possui patrimônio próprio e, à época da dissolução da sociedade conjugal, tem direito à metade dos bens adquiridos pelo casal, a título oneroso, na constância do casamento.',
  'justificativa': 'O regime de participação final nos aqüestos é regulamentado pelo Art. 1.672 do Código Civil Brasileiro.',
  'artigos': 'Art. 1.672',
  'contexto': 'No regime de participação final nos aqüestos, cada cônjuge possui patrimônio próprio, consoante disposto no artigo seguinte, e lhe cabe, à época da dissolução da sociedade conjugal, direito à metade dos bens adquiridos pelo casal, a título oneroso, na constância do casamento.',
  'ref_pergunta': '16. Meu filho vai completar 18 anos, ele vai deixar de receber a pensão alimentícia?',
  'ref_resposta': 'Não. Como respondido anteriormente, o simples fato de o filho completar 18 anos não dá direito ao alimentante de que pare de pagar a pens

## Loop

In [ ]:
output_list = []
for i in tqdm(range(30), desc="Processando"):
    try:
        qa_json = call_qa_gen_agent(iv_articles, v_articles, faq_qa)
        output_list.append(qa_json)
        print(qa_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        continue

llm1_df = pd.DataFrame(output_list)
llm1_df.to_csv("LLM1_QA_generation.csv", index=False)

Processando:   3%|▎         | 1/30 [00:03<01:36,  3.33s/it]

[{'pergunta': 'Quem não pode ser tutor?', 'resposta': 'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.', 'justificativa': 'Essas pessoas não podem ser tutores porque não têm condições de exercer a tutela de forma adequada.', 'artigos': 'Art. 1.735', 'contexto': 'Art. 1.735. Não podem ser tutores e serão exonerados da tutela, caso a exerçam: I - aqueles que não tiverem a livre administração de seus bens; II - ...; III - ...; IV - ...; V - ...; VI - ...', 'ref_pergunta': '4. O cônjuge tem direito à pensão após o divórcio?', 'ref_resposta': 'Apesar de menos com o pedido de pensão pelo cônjuge após o divórcio, é possível sim realizar o pedido. Porém, isso só irá acontecer caso fique demonstrado que não possui condições de trabalhar, seja por ter uma doença, ou seja por questões do próprio casamento (o ex-marido não deixava trabalhar e a esposa dependia dele). Condições da idade também s

Processando:   7%|▋         | 2/30 [00:10<02:37,  5.64s/it]

[{'pergunta': 'Como se prova o casamento celebrado no Brasil?', 'resposta': 'Pela certidão do registro.', 'justificativa': 'O casamento celebrado no Brasil prova-se pela certidão do registro.', 'artigos': 'Art. 1.543', 'contexto': 'Art. 1.543. O casamento celebrado no Brasil prova-se pela certidão do registro.', 'ref_pergunta': '19. Como funciona uma audiência de pensão alimentícia?', 'ref_resposta': 'Nesse tipo de situação presa-se pela conciliação. Dessa forma, as partes terão a oportunidade de conversar, e buscar conciliar. Dependendo da fase em que estiver o processo, podem ser apresentadas provas, documentos, ou ainda, ouvida testemunhas.', 'ref_artigos': "['Art. 1.543. O casamento celebrado no Brasil prova-se pela certidão do registro.\\n\\nParágrafo único. Justificada a falta ou perda do registro civil, é admissível qualquer outra espécie de prova.\\n\\n  ', 'Art. 1.544. O casamento de brasileiro, celebrado no estrangeiro, perante as respectivas autoridades ou os cônsules brasil

Processando:  10%|█         | 3/30 [00:36<06:40, 14.83s/it]

...

[{'pergunta': 'Quais são as regras que se aplicam ao exercício da curatela?', 'resposta': 'As regras que se aplicam ao exercício da tutela.', 'justificativa': 'A curatela segue as mesmas regras da tutela, com algumas restrições.', 'artigos': 'Art. 1.781', 'contexto': 'As regras a respeito do exercício da tutela aplicam-se ao da curatela, com a restrição do art. 1.772 e as desta Seção.', 'ref_pergunta': '19. Como funciona uma audiência de pensão alimentícia?', 'ref_resposta': 'Nesse tipo de situação presa-se pela conciliação. Dessa forma, as partes terão a oportunidade de conversar, e buscar conciliar. Dependendo da fase em que estiver o processo, podem ser apresentadas provas, documentos, ou ainda, ouvida testemunhas.', 'ref_artigos': "['Art. 1.781. As regras a respeito do exercício da tutela aplicam-se ao da curatela, com a restrição do art. 1.772 e as desta Seção.\\n\\n  ', 'Art. 1.782. A interdição do pródigo só o privará de, sem curador, emprestar, transigir, dar quitação, alienar,

Processando:  97%|█████████▋| 29/30 [13:43<00:32, 32.69s/it]

[{'pergunta': 'Quais bens são comunicáveis no regime de comunhão parcial?', 'resposta': 'Os bens adquiridos na constância do casamento por título oneroso, os bens adquiridos por fato eventual, os bens adquiridos por doação, herança ou legado em favor de ambos os cônjuges, as benfeitorias em bens particulares de cada cônjuge e os frutos dos bens comuns ou particulares.', 'justificativa': 'Esses bens são comunicáveis porque foram adquiridos durante o casamento e são considerados como pertencentes ao casal.', 'artigos': 'Art. 1.660', 'contexto': 'Art. 1.660. Entram na comunhão: I - os bens adquiridos na constância do casamento por título oneroso, ainda que só em nome de um dos cônjuges; II - os bens adquiridos por fato eventual, com ou sem o concurso de trabalho ou despesa anterior; III - os bens adquiridos por doação, herança ou legado, em favor de ambos os cônjuges; IV - as benfeitorias em bens particulares de cada cônjuge; V - os frutos dos bens comuns, ou dos particulares de cada cônj

Processando: 100%|██████████| 30/30 [14:14<00:00, 28.49s/it]

[{'pergunta': 'Quais bens são excluídos da comunhão parcial?', 'resposta': 'Os bens que cada cônjuge possuir ao casar, e os que lhe sobrevierem, na constância do casamento, por doação ou sucessão, e os sub-rogados em seu lugar.', 'justificativa': 'Os bens excluídos da comunhão parcial são aqueles que cada cônjuge possuir ao casar, e os que lhe sobrevierem, na constância do casamento, por doação ou sucessão, e os sub-rogados em seu lugar.', 'artigos': 'Art. 1.659', 'contexto': 'Art. 1.659. Excluem-se da comunhão: I - os bens que cada cônjuge possuir ao casar, e os que lhe sobrevierem, na constância do casamento, por doação ou sucessão, e os sub-rogados em seu lugar;', 'ref_pergunta': 'Há alguma exceção onde um enteado possa receber a herança?', 'ref_resposta': 'Sim, no caso de padrastros e madrastas que podem ser mais presentes que os próprios pais/mães, ocorre a filiação socioafetiva, que gera a possibilidade do enteado se tornar herdeiro legítimo.A socioafetividade é tratada na Lei n.

In [ ]:
llm1_df

,0,1,2,3,4,5,6,7,8,9
0,"{'pergunta': 'Quem não pode ser tutor?', 'resp...",{'pergunta': 'O que acontece quando o herdeiro...,{'pergunta': 'Como o herdeiro pode aceitar a h...,{'pergunta': 'O herdeiro pode renunciar a hera...,{'pergunta': 'O que acontece se o herdeiro não...,{'pergunta': 'O herdeiro pode aceitar ou renun...,None,None,None,None
1,{'pergunta': 'Como se prova o casamento celebr...,{'pergunta': 'O que acontece se o casamento fo...,{'pergunta': 'Como se prova a celebração legal...,{'pergunta': 'Como se aceita ou renuncia a her...,{'pergunta': 'O que acontece se o herdeiro ren...,{'pergunta': 'É possível aceitar ou renunciar ...,None,None,None,None
2,{'pergunta': 'Quais bens são excluídos da comu...,{'pergunta': 'O que acontece com os frutos dos...,{'pergunta': 'Como é feita a administração dos...,{'pergunta': 'Quem pode demandar o reconhecime...,{'pergunta': 'O que acontece com os bens da he...,None,None,None,None,None
3,{'pergunta': 'Quais são os direitos dos filhos...,{'pergunta': 'Quais são as presunções de conce...,{'pergunta': 'Quem pode contestar a paternidad...,"{'pergunta': 'Como se prova a filiação?', 'res...",{'pergunta': 'Quais são os tipos de testamento...,None,None,None,None,None
4,{'pergunta': 'Quem tem o direito de nomear tut...,{'pergunta': 'O que acontece se os pais não no...,{'pergunta': 'Quais são os herdeiros necessári...,"{'pergunta': 'O que é a legítima?', 'resposta'...",{'pergunta': 'Um menor abandonado pode ter um ...,None,None,None,None,None
5,{'pergunta': 'Quais são os tipos de parentesco...,{'pergunta': 'Como são contados os graus de pa...,{'pergunta': 'Quais são os testamentos especia...,"{'pergunta': 'O que é afinidade?', 'resposta':...",{'pergunta': 'A afinidade se extingue com a di...,None,None,None,None,None
6,{'pergunta': 'Quem tem o poder familiar sobre ...,{'pergunta': 'O que acontece com o poder famil...,{'pergunta': 'Quem pode ser excluído da sucess...,{'pergunta': 'Como é declarada a exclusão do h...,{'pergunta': 'Quais são os efeitos da exclusão...,None,None,None,None,None
7,{'pergunta': 'Quem pode promover a interdição?...,{'pergunta': 'Quais são os casos em que o Mini...,{'pergunta': 'Quem é o defensor do suposto inc...,{'pergunta': 'Quais são os limites da curatela...,{'pergunta': 'Quem é o curador do cônjuge ou c...,{'pergunta': 'Quem pode demandar o reconhecime...,None,None,None,None
8,{'pergunta': 'Quais são as condições para cess...,{'pergunta': 'Quais são as razões para cessar ...,{'pergunta': 'Quem tem direito à herança em ca...,{'pergunta': 'Quais são os direitos da companh...,{'pergunta': 'Quais são os limites da disposiç...,None,None,None,None,None
9,{'pergunta': 'Quem exerce o poder familiar em ...,{'pergunta': 'O que acontece com o poder famil...,{'pergunta': 'Quem pode dispor de seus bens po...,{'pergunta': 'O que acontece com a legítima do...,{'pergunta': 'Quanto tempo tem um herdeiro par...,None,None,None,None,None


In [ ]:
valores = llm1_df.values.reshape(-1)
valores = valores[valores != None]
llm1_df = pd.DataFrame(valores.tolist())
llm1_df.to_csv("LLM1_QA_generation.csv", index=False)

In [ ]:
llm1_df

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos
0,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...,Essas pessoas não podem ser tutores porque não...,Art. 1.735,Art. 1.735. Não podem ser tutores e serão exon...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
1,O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.,A aceitação da herança implica a transmissão d...,Art. 1.804,"Art. 1.804. Aceita a herança, torna-se definit...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
2,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...,A aceitação da herança pode ser feita de forma...,Art. 1.805,"Art. 1.805. A aceitação da herança, quando exp...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
3,O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança.","A renúncia da herança é possível, mas deve ser...",Art. 1.806,Art. 1.806. A renúncia da herança deve constar...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
4,O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...,Se o herdeiro não se pronunciar sobre a heranç...,Art. 1.807,Art. 1.807. O interessado em que o herdeiro de...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
...,...,...,...,...,...,...,...,...
169,Quais bens são excluídos da comunhão parcial?,"Os bens que cada cônjuge possuir ao casar, e o...",Os bens excluídos da comunhão parcial são aque...,Art. 1.659,Art. 1.659. Excluem-se da comunhão: I - os ben...,Há alguma exceção onde um enteado possa recebe...,"Sim, no caso de padrastros e madrastas que pod...","['Art. 1.658. No regime de comunhão parcial, c..."
170,Quais bens entram na comunhão parcial?,Os bens adquiridos na constância do casamento ...,Os bens que entram na comunhão parcial são aqu...,Art. 1.660,Art. 1.660. Entram na comunhão: I - os bens ad...,Há alguma exceção onde um enteado possa recebe...,"Sim, no caso de padrastros e madrastas que pod...","['Art. 1.658. No regime de comunhão parcial, c..."
171,Quem administra o patrimônio comum?,Qualquer dos cônjuges.,A administração do patrimônio comum compete a ...,Art. 1.663,Art. 1.663. A administração do patrimônio comu...,Há alguma exceção onde um enteado possa recebe...,"Sim, no caso de padrastros e madrastas que pod...","['Art. 1.658. No regime de comunhão parcial, c..."
172,Como é feita a publicação do testamento?,Com citação dos herdeiros legítimos.,"Morto o testador, publicar-se-á em juízo o tes...",Art. 1.877,"Art. 1.877. Morto o testador, publicar-se-á em...",Há alguma exceção onde um enteado possa recebe...,"Sim, no caso de padrastros e madrastas que pod...","['Art. 1.658. No regime de comunhão parcial, c..."


## Join Outputs and Clean repeated questions

Devido a um erro relacionado a amostragem de dados com pandas, foram executados por completo dois loops na LLM1. No primeiro loop a pergunta de exemplo se repetiu para todos os prompts. Entendendo que esse erro não afeta diretamente os resultados podemos juntar os dois dataframes e obter mais perguntas.

In [ ]:
llm1_df = pd.read_csv("/content/LLM1_QA_generation.csv")

llm1_try1_df = pd.read_csv("/content/LLM1_QA_generation_try1_same_example.csv")
llm1_try1_df

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos
0,Quais são as responsabilidades dos tutores em ...,Os tutores não podem conservar em seu poder di...,Os tutores devem aplicar os valores dos tutela...,Art. 1.753,"§ 1 o Se houver necessidade, os objetos de our...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.753. Os tutores não podem conservar e...
1,Quais são as condições para se retirar valores...,Os valores que existirem em estabelecimento ba...,Os valores dos tutelados devem ser aplicados d...,Art. 1.754,I - para as despesas com o sustento e educação...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.753. Os tutores não podem conservar e...
2,Quais são as condições para se anular uma part...,A partilha é anulável pelos vícios e defeitos ...,A partilha pode ser anulada se houver vícios o...,Art. 2.027,"A partilha, uma vez feita e julgada, só é anul...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.753. Os tutores não podem conservar e...
3,Quais são os prazos para se anular uma partilha?,O direito de anular a partilha se extingue em ...,O prazo para se anular uma partilha é de um ano.,Art. 2.027,Parágrafo único. Extingue-se em um ano o direi...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.753. Os tutores não podem conservar e...
4,Quais são os direitos dos filhos havidos ou nã...,"Os filhos, havidos ou não da relação de casame...",O art. 1.596 do Código Civil Brasileiro estabe...,Art. 1.596,"Art. 1.596. Os filhos, havidos ou não da relaç...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.596. Os filhos, havidos ou não da rel..."
...,...,...,...,...,...,...,...,...
165,Quais documentos são necessários para o requer...,Certidão de nascimento ou documento equivalent...,Os documentos necessários para o requerimento ...,Art. 1.525,O requerimento de habilitação para o casamento...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
166,Quem pode impugnar a habilitação para o casame...,"O oficial, o Ministério Público ou terceiro po...","Caso haja impugnação do oficial, do Ministério...",Art. 1.526,"Caso haja impugnação do oficial, do Ministério...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
167,Quais são os legitimados a suceder?,As pessoas nascidas ou já concebidas no moment...,Os legitimados a suceder estão previstos nos A...,"Art. 1.798, Art. 1.799",Legitimam-se a suceder as pessoas nascidas ou ...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
168,Quem não pode ser nomeado herdeiro ou legatário?,"A pessoa que, a rogo, escreveu o testamento, n...",Os impedidos de serem nomeados herdeiros ou le...,Art. 1.801,Não podem ser nomeados herdeiros nem legatários:,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...


In [ ]:
llm1_df = pd.concat([llm1_df, llm1_try1_df], ignore_index=True)
llm1_df

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos
0,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...,Essas pessoas não podem ser tutores porque não...,Art. 1.735,Art. 1.735. Não podem ser tutores e serão exon...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
1,O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.,A aceitação da herança implica a transmissão d...,Art. 1.804,"Art. 1.804. Aceita a herança, torna-se definit...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
2,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...,A aceitação da herança pode ser feita de forma...,Art. 1.805,"Art. 1.805. A aceitação da herança, quando exp...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
3,O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança.","A renúncia da herança é possível, mas deve ser...",Art. 1.806,Art. 1.806. A renúncia da herança deve constar...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
4,O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...,Se o herdeiro não se pronunciar sobre a heranç...,Art. 1.807,Art. 1.807. O interessado em que o herdeiro de...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
...,...,...,...,...,...,...,...,...
339,Quais documentos são necessários para o requer...,Certidão de nascimento ou documento equivalent...,Os documentos necessários para o requerimento ...,Art. 1.525,O requerimento de habilitação para o casamento...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
340,Quem pode impugnar a habilitação para o casame...,"O oficial, o Ministério Público ou terceiro po...","Caso haja impugnação do oficial, do Ministério...",Art. 1.526,"Caso haja impugnação do oficial, do Ministério...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
341,Quais são os legitimados a suceder?,As pessoas nascidas ou já concebidas no moment...,Os legitimados a suceder estão previstos nos A...,"Art. 1.798, Art. 1.799",Legitimam-se a suceder as pessoas nascidas ou ...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...
342,Quem não pode ser nomeado herdeiro ou legatário?,"A pessoa que, a rogo, escreveu o testamento, n...",Os impedidos de serem nomeados herdeiros ou le...,Art. 1.801,Não podem ser nomeados herdeiros nem legatários:,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...


In [ ]:
llm1_df = llm1_df.drop_duplicates(subset=['pergunta'])
len(llm1_df)

296

In [ ]:
#len(llm1_df.drop_duplicates(subset=['resposta']))
#llm1_df[llm1_df['resposta'].duplicated(keep=False)].sort_values(by=['resposta'], ascending=False)
llm1_df = llm1_df.drop([171, 315, 278, 19, 225, 109]) # manual selection - similar questions based on equal answers
len(llm1_df)

290

In [ ]:
llm1_df = llm1_df.reset_index(drop=True)
llm1_df.to_csv("LLM1_QA_generation_clean.csv", index=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

# Step 1: Vectorization using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(llm1_df['pergunta'])

# Step 2: Clustering with K-means
num_clusters = 100  # Number of clusters (you can adjust this)
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Step 3: Assigning clusters and finding cluster centers
llm1_df['cluster'] = kmeans.labels_

In [ ]:
similar_df = pd.DataFrame(llm1_df.groupby('cluster')['pergunta'].agg(lambda x: list(x)))
similar_df.to_csv("similar_questions.csv")
similar_df

,pergunta
cluster,
0,[O que acontece se o casamento for celebrado n...
1,[Quem pode promover a ação de nulidade de casa...
2,[Quais são os efeitos da exclusão do herdeiro ...
3,[Quais bens são excluídos da comunhão universa...
4,[Quem pode dispor de seus bens por testamento?...
...,...
95,[Como se faz a partilha de uma herança quando ...
96,[Quem pode requerer partilha imediata ou devol...
97,[Quais são os direitos dos colaterais na suces...


In [ ]:
# Após analise manual - cheguei a um conjunto de clusters que podem ter perguntas repetidas
#cluster2look = [2, 4, 6, 8, 10, 11, 12, 13, 18, 19, 20, 21, 25, 26, 27, 31, 33, 34, 37, 40, 41, 45, 46, 56, 66, 82, 87, 91]
#llm1_df[llm1_df['cluster']==cluster2look[27]]

llm1_df = llm1_df.drop([242, 118, 286, 180, 47, 106, 216, 231, 11, 10, 256, 251, 36]).reset_index(drop=True)
len(llm1_df)

277

In [ ]:
llm1_df.to_csv("LLM1_QA_generation_clean2.csv", index=False)

# LLM 2: Avaliador de Qualidade da Questão

In [ ]:
class QUESTION_SCORE_JSON(BaseModel):
    pergunta: str = Field(description="questão avaliada")
    score: str = Field(description="score de qualidade dado a questão")
    justificativa: str = Field(description="razão pelo qual a questão recebeu esse score")
    artigos_id: str = Field(description="Id dos artigos que respondem completamente a questão")

output_parser = JsonOutputParser(pydantic_object=QUESTION_SCORE_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar.
Este TCC consiste em um FAQ para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil Brasileiro.
Sua tarefa então é dado uma pergunta do FAQ e um conjunto de artigos do Código Civil, determinar um score para o quanto essa pergunta pode ser respondida pelos artigos.

Para isso use a seguinte fórmula:
score 0 - Os artigos e a pergunta são completamente diferentes
score 1 - Os artigos não a respondem, mas falam do mesmo assunto
score 2 - Os artigos respondem parcialmente a pergunta
score 3 - Os artigos respondem completamente a pergunta

Para completar sua saída indique o id dos artigos que respondem a pergunta completamente.

Vamos começar a avaliação:
Artigos: {artigos}
Pergunta do FAQ: {pergunta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "pergunta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_question_score_agent(llm1_sample):
    out = chain.invoke({"artigos": llm1_sample['ref_artigos'], "pergunta": llm1_sample['pergunta']})
    return out

# test
call_question_score_agent(llm1_df.iloc[0])

{'pergunta': 'Quem não pode ser tutor?',
 'score': '3',
 'justificativa': 'Os artigos respondem completamente a pergunta, listando as pessoas que não podem ser tutores.',
 'artigos_id': 'Art. 1.735'}

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        qscore_json = call_question_score_agent(llm1_sample)
        output_list.append(qscore_json)
        print(qscore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        continue

llm2_df = pd.DataFrame(output_list)
llm2_df.to_csv("LLM2_Q_score.csv", index=False)

  0%|          | 1/277 [00:00<03:36,  1.28it/s]

{'pergunta': 'Quem não pode ser tutor?', 'score': '3', 'justificativa': 'Os artigos respondem completamente a pergunta, listando as pessoas que não podem ser tutores.', 'artigos_id': 'Art. 1.735'}


  1%|          | 2/277 [00:01<04:19,  1.06it/s]

{'pergunta': 'O que acontece quando o herdeiro aceita a herança?', 'score': '3', 'justificativa': 'Os artigos 1.804, 1.805 e 1.806 do Código Civil Brasileiro respondem completamente à pergunta, explicando que a aceitação da herança torna-se definitiva a sua transmissão ao herdeiro, desde a abertura da sucessão, e como ela pode ser feita de forma expressa ou tácita.', 'artigos_id': 'Art. 1.804, Art. 1.805, Art. 1.806'}


  1%|          | 3/277 [00:02<04:22,  1.04it/s]

...

{'pergunta': 'Quais são os legitimados a suceder?', 'score': '3', 'justificativa': 'Os artigos 1.798, 1.799 e 1.800 do Código Civil Brasileiro respondem completamente à pergunta, enumerando os legitimados a suceder.', 'artigos_id': 'Art. 1.798, Art. 1.799, Art. 1.800'}


100%|█████████▉| 276/277 [1:37:01<00:21, 21.73s/it]

{'pergunta': 'Quem não pode ser nomeado herdeiro ou legatário?', 'score': '3', 'justificativa': 'Os artigos respondem completamente a pergunta, pois o Art. 1.801 lista as pessoas que não podem ser nomeadas herdeiras ou legatárias.', 'artigos_id': 'Art. 1.801'}


100%|██████████| 277/277 [1:37:23<00:00, 21.10s/it]

{'pergunta': 'O que acontece se o herdeiro esperado não for concebido?', 'score': '3', 'justificativa': 'O Art. 1.800, § 4º, do Código Civil Brasileiro estabelece que, se decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados caberão aos herdeiros legítimos.', 'artigos_id': 'Art. 1.800, § 4º'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
llm2_df = pd.DataFrame(output_list)
llm2_df.to_csv("LLM2_Q_score_v2.csv", index=False)

In [ ]:
! cp LLM2_Q_score_v2.csv /content/drive/MyDrive/LLM2_Q_score_v2.csv

In [ ]:
llm2_df['score'].value_counts()

score
3    267
2      8
1      1
0      1
Name: count, dtype: int64

# LLM 3 - Avaliador da Qualidade da Resposta

In [ ]:
class ANSWER_SCORE_JSON(BaseModel):
    resposta_original: str = Field(description="resposta avaliada")
    score: str = Field(description="score de qualidade dado a resposta")
    justificativa: str = Field(description="razão pelo qual a resposta recebeu esse score")
    artigos_id: str = Field(description="Id dos artigos que contêm a resposta")

output_parser = JsonOutputParser(pydantic_object=ANSWER_SCORE_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar. Este TCC consiste em um FAQ para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil Brasileiro.
Sua tarefa então é dado uma resposta do FAQ e um conjunto de artigos do Código Civil, determinar um score para o quanto essa resposta está aderente aos artigos e pode ser obtida a partir deles.

Para isso use a seguinte fórmula:
score 0 - A resposta não está contida nos artigos
score 1 - A resposta está contida nos artigos, mas não está de acordo com eles
score 2 - A resposta está contida nos artigos e está de acordo com eles.
score 3 - A resposta é uma cópia de um dos trechos dos artigos.

Em sua justificativa indique o porquê de você acreditar que a resposta está contida ou não nos artigos.
Para completar sua saída indique o id dos artigos nas quais a resposta foi retirada.

Vamos começar a avaliação:
Artigos: {artigos}
Resposta do FAQ: {resposta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "resposta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_answer_score_agent(llm1_sample):
    out = chain.invoke({"artigos": llm1_sample['ref_artigos'], "resposta": llm1_sample['resposta']})
    return out

# test
call_answer_score_agent(llm1_df.iloc[118])

{'resposta_original': 'O edital é afixado durante quinze dias nas circunscrições do Registro Civil de ambos os nubentes e publicado na imprensa local, se houver.',
 'score': '2',
 'justificativa': 'A resposta está contida no Art. 1.527 do Código Civil Brasileiro, que estabelece que o edital deve ser afixado durante quinze dias nas circunscrições do Registro Civil de ambos os nubentes e publicado na imprensa local, se houver.',
 'artigos_id': 'Art. 1.527'}

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        ascore_json = call_answer_score_agent(llm1_sample)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

  0%|          | 1/277 [00:01<05:25,  1.18s/it]

{'resposta_original': 'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.', 'score': '2', 'justificativa': 'A resposta está contida nos artigos e está de acordo com eles. O art. 1.735 do Código Civil Brasileiro lista os impedimentos para ser tutor, incluindo aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.', 'artigos_id': 'Art. 1.735'}


  1%|          | 2/277 [00:02<04:44,  1.03s/it]

{'resposta_original': 'A transmissão da herança torna-se definitiva.', 'score': '2', 'justificativa': "A resposta está contida no Art. 1.804 do Código Civil Brasileiro, que diz: 'Aceita a herança, torna-se definitiva a sua transmissão ao herdeiro, desde a abertura da sucessão.'", 'artigos_id': 'Art. 1.804'}


  1%|          | 3/277 [00:03<04:28,  1.02it/s]

...

{'resposta_original': 'As pessoas nascidas ou já concebidas no momento da abertura da sucessão, os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão, as pessoas jurídicas e as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação.', 'score': '2', 'justificativa': 'A resposta está contida nos artigos 1.798, 1.799 e 1.799 do Código Civil Brasileiro, que tratam sobre a legitimidade para suceder. A resposta apresenta uma lista de pessoas que podem ser chamadas a suceder, que está de acordo com os artigos mencionados.', 'artigos_id': 'Art. 1.798, Art. 1.799'}


100%|█████████▉| 276/277 [1:39:57<00:23, 23.10s/it]

{'resposta_original': 'A pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos, as testemunhas do testamento, o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos, e o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou aprovar o testamento.', 'score': '3', 'justificativa': 'A resposta é uma cópia de um dos trechos dos artigos, especificamente do Art. 1.801 do Código Civil Brasileiro.', 'artigos_id': 'Art. 1.801'}


100%|██████████| 277/277 [1:40:22<00:00, 21.74s/it]

{'resposta_original': 'Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'score': '2', 'justificativa': 'A resposta está contida no art. 1.800, § 4º do Código Civil Brasileiro, que estabelece que se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'artigos_id': 'Art. 1.800, § 4º'}


In [ ]:
llm3_df = pd.DataFrame(output_list)
llm3_df.to_csv("LLM3_A_score_v2.csv", index=False)

In [ ]:
! cp LLM3_A_score_v2.csv /content/drive/MyDrive/LLM3_A_score_v2.csv

In [ ]:
llm3_df['score'].value_counts()

score
2    254
3     16
0      5
1      2
Name: count, dtype: int64

# LLM 4 - Avaliador da Resposta em Relação a Pergunta

In [ ]:
class ANSWER_RESPONSIVENESS_JSON(BaseModel):
    justificativa: str = Field(description="razão pelo qual a resposta recebeu esse score")
    score: str = Field(description="score de responsividade dado a resposta")
    resposta_mais_completa: str = Field(description="resposta mais completa que a resposta avaliada")
    pergunta_original: str = Field(description="pergunta avaliada")
    resposta_original: str = Field(description="resposta avaliada")

output_parser = JsonOutputParser(pydantic_object=ANSWER_RESPONSIVENESS_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar. Este TCC consiste em um FAQ para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil.
Sua tarefa então é avaliar por meio de um score o quanto a resposta do FAQ responde a pergunta feita. Use os artigos como referência para responder.
Seja critíco. Você pode prestar atenção a detalhes como se de fato a resposta responde a COMO, QUEM, QUANTO, O QUE, ...

Antes de determinar o score formule uma resposta mais completa para a pergunta com base nos artigos que receber.
Desse modo, será mais fácil avaliar a completude da resposta em relação a pergunta.

Para o score de responsividade use a seguinte fórmula:
score 0 - A resposta não responde a pergunta.
score 1 - A resposta responde parcialmente a pergunta
score 2 - A resposta responde a pergunta completamente
score 3 - A resposta responde a pergunta e fornece informações extras.
O score SEMPRE será apenas o número.

Sua justificativa deve concordar com o score fornecido e sua justificativa deve apontar as possíveis ausências de informação na resposta.

Em resumo produza a saída em formato JSON com as seguintes chaves:
score
justificativa
resposta mais completa
pergunta original
resposta original

Vamos começar a avaliação:
Artigos: {artigos}
Pergunta do FAQ: {pergunta}
Resposta do FAQ: {resposta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "pergunta", "resposta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
llm1_df = pd.read_csv("/content/LLM1_QA_generation_clean2.csv")

In [ ]:
len(llm1_df)

277

In [ ]:
def call_answer_responsiveness_score(llm1_sample):
    out = chain.invoke(
        {"artigos": llm1_sample['ref_artigos'],
         "pergunta": llm1_sample['pergunta'],
         "resposta": llm1_sample['resposta']})
    return out

# test
call_answer_responsiveness_score(llm1_df.iloc[0])

{'justificativa': 'A resposta não menciona todos os casos em que alguém não pode ser tutor, como aqueles que exercerem função pública incompatível com a boa administração da tutela, ou que tiverem que fazer valer direitos contra o menor.',
 'score': '1',
 'resposta_mais_completa': 'Não podem ser tutores e serão exonerados da tutela, caso a exerçam: aqueles que não tiverem a livre administração de seus bens; aqueles que, no momento de lhes ser deferida a tutela, se acharem constituídos em obrigação para com o menor, ou tiverem que fazer valer direitos contra este, e aqueles cujos pais, filhos ou cônjuges tiverem demanda contra o menor; os inimigos do menor, ou de seus pais, ou que tiverem sido por estes expressamente excluídos da tutela; os condenados por crime de furto, roubo, estelionato, falsidade, contra a família ou os costumes, tenham ou não cumprido pena; as pessoas de mau procedimento, ou falhas em probidade, e as culpadas de abuso em tutorias anteriores; e aqueles que exercerem

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        ascore_json = call_answer_responsiveness_score(llm1_sample)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

llm4_df = pd.DataFrame(output_list)
llm4_df.to_csv("LLM4_A_resp_score.csv", index=False)

  0%|          | 1/277 [00:02<12:19,  2.68s/it]

{'justificativa': 'A resposta original apresenta uma lista parcial de pessoas que não podem ser tutores, mas não apresenta a lista completa e detalhada prevista no Art. 1.735 do Código Civil Brasileiro.', 'score': '1', 'resposta_mais_completa': 'De acordo com o Art. 1.735 do Código Civil Brasileiro, não podem ser tutores e serão exonerados da tutela: aqueles que não tiverem a livre administração de seus bens; aqueles que, no momento de lhes ser deferida a tutela, se acharem constituídos em obrigação para com o menor, ou tiverem que fazer valer direitos contra este, e aqueles cujos pais, filhos ou cônjuges tiverem demanda contra o menor; os inimigos do menor, ou de seus pais, ou que tiverem sido por estes expressamente excluídos da tutela; os condenados por crime de furto, roubo, estelionato, falsidade, contra a família ou os costumes, tenham ou não cumprido pena; as pessoas de mau procedimento, ou falhas em probidade, e as culpadas de abuso em tutorias anteriores; e aqueles que exercer

  1%|          | 2/277 [00:20<51:59, 11.34s/it]

{'justificativa': 'A resposta do FAQ apenas menciona que a transmissão da herança torna-se definitiva, mas não explica como isso ocorre ou quais são as consequências dessa aceitação. Além disso, não há menção à necessidade de declaração escrita ou tácita da aceitação, como estabelecido nos artigos 1.804 e 1.805 do Código Civil.', 'score': '1', 'resposta_mais_completa': 'Quando o herdeiro aceita a herança, a transmissão torna-se definitiva, desde a abertura da sucessão (Art. 1.804). A aceitação pode ser expressa, por meio de declaração escrita, ou tácita, por meio de atos próprios da qualidade de herdeiro (Art. 1.805). Com a aceitação, o herdeiro assume a responsabilidade pela herança e pode começar a exercer seus direitos e cumprir suas obrigações.', 'pergunta_original': 'O que acontece quando o herdeiro aceita a herança?', 'resposta_original': 'A transmissão da herança torna-se definitiva.'}


  1%|          | 3/277 [00:41<1:13:08, 16.02s/it]

...

100%|█████████▉| 276/277 [1:54:32<00:27, 27.21s/it]

Erro na iteração 275: Invalid json output: Avaliação do FAQ:

Pergunta Original: Quem não pode ser nomeado herdeiro ou legatário?
Resposta Original: A pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos, as testemunhas do testamento, o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos, e o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou aprovar o testamento.

Resposta Mais Completa: De acordo com o Art. 1.801 do Código Civil Brasileiro, não podem ser nomeados herdeiros nem legatários: a pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos; as testemunhas do testamento; o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos; e o tabelião, civil ou militar, ou o comandante ou escrivão, per

100%|██████████| 277/277 [1:55:01<00:00, 24.91s/it]

{'justificativa': 'A resposta do FAQ responde parcialmente à pergunta, pois apenas menciona o que acontece se o herdeiro esperado não for concebido, mas não explica o contexto em que isso ocorre. Além disso, a resposta não menciona que os bens serão confiados a um curador nomeado pelo juiz após a liquidação ou partilha.', 'score': '1', 'resposta_mais_completa': 'Se o herdeiro esperado não for concebido, os bens da herança serão confiados, após a liquidação ou partilha, a um curador nomeado pelo juiz. Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'pergunta_original': 'O que acontece se o herdeiro esperado não for concebido?', 'resposta_original': 'Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.'}


AttributeError: 'str' object has no attribute 'keys'

In [ ]:
for idx, output in enumerate(output_list):
    if output == 'ERROR':
        print(idx)
        ascore_json = call_answer_responsiveness_score(llm1_df.iloc[idx])
        print(ascore_json)
        print()
        output_list[idx] = ascore_json


7
{'justificativa': 'A resposta do FAQ é parcialmente correta, pois menciona a necessidade de registro do casamento no estrangeiro, mas não especifica as condições e prazos para tal registro.', 'score': '1', 'resposta_mais_completa': 'Se o casamento for celebrado no estrangeiro, perante as respectivas autoridades ou os cônsules brasileiros, deverá ser registrado em cento e oitenta dias, a contar da volta de um ou de ambos os cônjuges ao Brasil, no cartório do respectivo domicílio, ou, em sua falta, no 1º Ofício da Capital do Estado em que passarem a residir (Art. 1.544 do Código Civil).', 'pergunta_original': 'O que acontece se o casamento for celebrado no estrangeiro?', 'resposta_original': 'Deve ser registrado em cento e oitenta dias no cartório do respectivo domicílio.'}

8
{'justificativa': 'A resposta responde parcialmente à pergunta, pois apenas menciona a certidão do Registro Civil como prova da celebração legal do casamento, mas não considera outras possibilidades de prova, com

In [ ]:
llm4_df = pd.DataFrame(output_list)
llm4_df.to_csv("LLM4_A_resp_score.csv", index=False)

In [ ]:
!cp LLM4_A_resp_score.csv /content/drive/MyDrive/LLM4_A_resp_score.csv

# LLM 5 - Pergunta + Resposta x Contexto (Completude)


In [ ]:
class ANSWER_COMPLETENESS_JSON(BaseModel):
    pergunta_original: str = Field(description="pergunta avaliada")
    resposta_original: str = Field(description="resposta avaliada")
    resposta_mais_completa: str = Field(description="resposta mais completa que a resposta avaliada")
    score: str = Field(description="score de completude dado a resposta")
    justificativa: str = Field(description="razão pelo qual a resposta recebeu esse score")

output_parser = JsonOutputParser(pydantic_object=ANSWER_COMPLETENESS_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar. Este TCC consiste em um FAQ para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil Brasileiro.
Sua tarefa então é avaliar por meio de um score o quão abrangente ou completa está a resposta em relação ao artigos de referência.

Use a seguinte fórmula:
score 0 - A resposta não responde a pergunta em relação aos artigos
score 1 - A resposta responde a pergunta, mas não em relação aos artigos
score 2 - A resposta responde a pergunta, mas em relação aos artigos a resposta está incompleta
score 3 - A resposta responde a pergunta e está completa em relação aos artigos

Para ajudar o aluno no TCC, além da avaliação forneça uma resposta mais completa que a resposta do FAQ.

Vamos começar a avaliação:
Artigos: {artigos}
Pergunta do FAQ: {pergunta}
Resposta do FAQ: {resposta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "pergunta", "resposta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
llm1_df.iloc[0]['resposta']

'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.'

In [ ]:
def call_answer_completeness_score(llm1_sample):
    out = chain.invoke(
        {"artigos": llm1_sample['ref_artigos'],
         "pergunta": llm1_sample['pergunta'],
         "resposta": llm1_sample['resposta']})
    return out

# test
call_answer_completeness_score(llm1_df.iloc[0])

{'pergunta_original': 'Quem não pode ser tutor?',
 'resposta_original': 'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.',
 'resposta_mais_completa': 'De acordo com o Art. 1.735 do Código Civil Brasileiro, não podem ser tutores: aqueles que não tiverem a livre administração de seus bens; aqueles que, no momento de lhes ser deferida a tutela, se acharem constituídos em obrigação para com o menor, ou tiverem que fazer valer direitos contra este, e aqueles cujos pais, filhos ou cônjuges tiverem demanda contra o menor; os inimigos do menor, ou de seus pais, ou que tiverem sido por estes expressamente excluídos da tutela; os condenados por crime de furto, roubo, estelionato, falsidade, contra a família ou os costumes, tenham ou não cumprido pena; as pessoas de mau procedimento, ou falhas em probidade, e as culpadas de abuso em tutorias anteriores; e aqueles que exercerem função pública i

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        ascore_json = call_answer_completeness_score(llm1_sample)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        continue

llm5_df = pd.DataFrame(output_list)
llm5_df.to_csv("LLM5_A_comp_score.csv", index=False)

  0%|          | 1/277 [00:01<07:33,  1.64s/it]

{'pergunta_original': 'Quem não pode ser tutor?', 'resposta_original': 'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.', 'resposta_mais_completa': 'De acordo com o Art. 1.735 do Código Civil Brasileiro, não podem ser tutores: aqueles que não tiverem a livre administração de seus bens; aqueles que, no momento de lhes ser deferida a tutela, se acharem constituídos em obrigação para com o menor, ou tiverem que fazer valer direitos contra este, e aqueles cujos pais, filhos ou cônjuges tiverem demanda contra o menor; os inimigos do menor, ou de seus pais, ou que tiverem sido por estes expressamente excluídos da tutela; os condenados por crime de furto, roubo, estelionato, falsidade, contra a família ou os costumes, tenham ou não cumprido pena; as pessoas de mau procedimento, ou falhas em probidade, e as culpadas de abuso em tutorias anteriores; e aqueles que exercerem função pública inc

  1%|          | 2/277 [00:02<06:22,  1.39s/it]

{'pergunta_original': 'O que acontece quando o herdeiro aceita a herança?', 'resposta_original': 'A transmissão da herança torna-se definitiva.', 'resposta_mais_completa': 'Quando o herdeiro aceita a herança, a transmissão torna-se definitiva, desde a abertura da sucessão, nos termos do art. 1.804 do Código Civil Brasileiro. Além disso, a aceitação da herança pode ser expressa ou tácita, devendo ser feita por declaração escrita ou resultar de atos próprios da qualidade de herdeiro, como previsto nos arts. 1.805 e 1.806 do Código Civil Brasileiro.', 'score': '3', 'justificativa': 'A resposta original é incompleta, pois não menciona a abertura da sucessão e as formas de aceitação da herança. A resposta mais completa fornece uma explicação mais detalhada e precisa, fazendo referência aos artigos relevantes do Código Civil Brasileiro.'}


  1%|          | 3/277 [00:04<06:06,  1.34s/it]

...

{'pergunta_original': 'Quais são os legitimados a suceder?', 'resposta_original': 'As pessoas nascidas ou já concebidas no momento da abertura da sucessão, os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão, as pessoas jurídicas e as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação.', 'resposta_mais_completa': 'De acordo com os artigos 1.798, 1.799 e 1.800 do Código Civil Brasileiro, os legitimados a suceder são: as pessoas nascidas ou já concebidas no momento da abertura da sucessão; os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão; as pessoas jurídicas; e as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação. Além disso, também podem ser chamados a suceder os herdeiros legítimos, nos casos em que não haja disposição testamentária.', 'score': '3', 'justificativa': 'A resposta orig

100%|█████████▉| 276/277 [1:46:08<00:24, 24.25s/it]

{'pergunta_original': 'Quem não pode ser nomeado herdeiro ou legatário?', 'resposta_original': 'A pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos, as testemunhas do testamento, o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos, e o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou aprovar o testamento.', 'resposta_mais_completa': 'De acordo com o Art. 1.801 do Código Civil Brasileiro, não podem ser nomeados herdeiros nem legatários: a pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos; as testemunhas do testamento; o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos; e o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou apr

100%|██████████| 277/277 [1:46:35<00:00, 23.09s/it]

{'pergunta_original': 'O que acontece se o herdeiro esperado não for concebido?', 'resposta_original': 'Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'resposta_mais_completa': 'De acordo com o Art. 1.800, § 4º do Código Civil Brasileiro, se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos. Além disso, é importante destacar que, se o herdeiro esperado nascer com vida, será deferida a sucessão, com os frutos e rendimentos relativos à deixa, a partir da morte do testador.', 'score': '3', 'justificativa': 'A resposta original está correta, mas a resposta mais completa fornece mais detalhes sobre a situação, incluindo a possibilidade de o herdeiro esperado nascer com vida e receber a sucessão.'}


In [ ]:
llm5_df = pd.DataFrame(output_list)
llm5_df.to_csv("LLM5_A_comp_score.csv", index=False)

In [ ]:
!cp LLM5_A_comp_score.csv /content/drive/MyDrive/LLM5_A_comp_score.csv

In [ ]:
!cp LLM5_A_comp_score.csv /content/drive/MyDrive/LLM5_A_comp_score.csv

In [ ]:
llm5_df['score'].value_counts()

score
3    165
2    112
Name: count, dtype: int64

# LLM 6 - Avaliador do Artigo falado -> Contexto é Verídico

Posso puxar o artigo citado do JSON

In [ ]:
def get_article_group(article_text, articles_library=iv_articles+v_articles):
    groups = []
    for group in articles_library:
        for article in group:
            if article_text in article:
                groups.append(group)
                break
    return groups

def get_articles(article_text, articles_library=iv_articles+v_articles):
    articles = []
    for group in articles_library:
        for article in group:
            if article_text in article:
                articles.append(article)
    return articles

def get_article_id(sample):
    arts = sample['artigos']
    arts = arts.split(',')
    for idx, art in enumerate(arts):
        if ('inciso' in art.lower()) or ('parágrafo' in art.lower()) or ('§' in art):
            arts.pop(idx)
            continue
        if ' e ' in art.lower():
            arts[idx] = art.split(' e ')[0].strip()
            arts.insert(idx+1, art.split(' e ')[1].strip())
        else:
            arts[idx] = art.strip()
    return arts

In [ ]:
class CONTEXT_REALITY_JSON(BaseModel):
    artigos_citados: str = Field(description="artigos citados")
    contexto_existe_no_artigo: str = Field(description="contexto existe no artigo, sim ou não?")
    contexto: str = Field(description="texto do contexto")
    score: str = Field(description="score de fidedignidade do contexto em relação ao artigo")
    justificativa: str = Field(description="razão pelo qual o contexto recebeu esse score")

output_parser = JsonOutputParser(pydantic_object=CONTEXT_REALITY_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar. Este TCC consiste em um FAQ para ajudar os demais alunos e a comunidade a entenderem melhor o Código Civil.
Sua tarefa então é avaliar se o aluno não mentiu ao dizer que sua resposta se baseia no trecho de um determinado artigo.
Para isso, fornecerei o texto correto do artigo citado pelo aluno e o texto do artigo falado pelo aluno o qual nominaremos "contexto".
Você deve responder se contexto existe no artigo.
Além disso indique o quão fidedigno é o contexto em relação ao artigo usando a seguinte fórmula:
score 0 - O contexto é completamente diferente do artigo.
score 1 - O contexto até aborda o mesmo tema do artigo, mas o artigo não existe no contexto.
score 2 - O artigo existe no contexto, mas foi modificado.
score 3 - O contexto referencia algum trecho do artigo de forma examente igual, isto é, com as mesmas palavras.

Justifique a sua resposta mostrando o raciocínio que você fez.

Em resumo produza a saída em formato JSON com as seguintes chaves:
artigos_citados
contexto_existe_no_artigo
contexto
score
justificativa

Vamos começar a avaliação.
Artigo Original: {artigos}
Contexto: {contexto}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "contexto"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_context_reliability_score(llm1_sample):

    arts = get_article_id(llm1_sample)
    art_text = '\n'.join(['\n'.join(get_articles(art)) for art in arts])

    out = chain.invoke(
        {"artigos": art_text,
         "contexto": llm1_sample['contexto']})
    return out

# test
call_context_reliability_score(llm1_df.iloc[13])

{'artigos_citados': 'Art. 1.824',
 'contexto_existe_no_artigo': 'Sim',
 'contexto': 'Art. 1.824: O herdeiro pode, em ação de petição de herança, demandar o reconhecimento de seu direito sucessório, para obter a restituição da herança, ou de parte dela, contra quem, na qualidade de herdeiro, ou mesmo sem título, a possua.',
 'score': '3',
 'justificativa': 'O contexto é idêntico ao artigo original, com as mesmas palavras e estrutura. Portanto, o contexto existe no artigo e é exatamente igual ao original.'}

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        ascore_json = call_context_reliability_score(llm1_sample)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

llm6_df = pd.DataFrame(output_list)
llm6_df.to_csv("LLM6_C_reab_score.csv", index=False)

  0%|          | 1/277 [00:01<07:08,  1.55s/it]

{'artigos_citados': 'Art. 1.735', 'contexto_existe_no_artigo': 'Sim', 'contexto': 'Art. 1.735. Não podem ser tutores e serão exonerados da tutela, caso a exerçam: I - aqueles que não tiverem a livre administração de seus bens; II - ...; III - ...; IV - ...; V - ...; VI - ...', 'score': '3', 'justificativa': 'O contexto apresentado pelo aluno é quase idêntico ao artigo original, com a mesma estrutura e mesmo inciso I. A única diferença é que os incisos II a VI foram omitidos no contexto, mas a estrutura e a redação são muito semelhantes.'}


  1%|          | 2/277 [00:04<10:57,  2.39s/it]

{'artigos_citados': 'Art. 1.804', 'contexto_existe_no_artigo': 'Sim', 'contexto': 'Art. 1.804. Aceita a herança, torna-se definitiva a sua transmissão ao herdeiro, desde a abertura da sucessão.', 'score': '3', 'justificativa': 'O contexto é idêntico ao artigo original, com as mesmas palavras e estrutura. Portanto, o score é 3, pois o contexto referencia o artigo de forma exatamente igual.'}


  1%|          | 3/277 [00:16<31:12,  6.83s/it]

...

{'artigos_citados': 'Art. 1.798', 'contexto_existe_no_artigo': 'Sim', 'contexto': 'Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.', 'score': '3', 'justificativa': 'O contexto é idêntico ao artigo 1.798, com as mesmas palavras, portanto, o score é 3, pois o contexto referencia o trecho do artigo de forma exatamente igual.'}


100%|█████████▉| 276/277 [52:25<00:11, 11.14s/it]

{'artigos_citados': 'Art. 1.801', 'contexto_existe_no_artigo': 'Sim', 'contexto': 'Não podem ser nomeados herdeiros nem legatários:', 'score': '2', 'justificativa': 'O contexto apresenta a mesma estrutura e começa com a mesma frase do artigo original, mas falta a enumeração dos incisos (I a IV) que especificam as pessoas que não podem ser nomeadas herdeiras ou legatárias. Portanto, o contexto existe no artigo, mas foi modificado.'}


100%|██████████| 277/277 [52:39<00:00, 11.41s/it]

{'artigos_citados': 'Art. 1.800', 'contexto_existe_no_artigo': 'Sim', 'contexto': 'Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'score': '3', 'justificativa': 'O contexto é idêntico ao § 4 o do art. 1.800, com as mesmas palavras e estrutura. Portanto, o contexto existe no artigo e é exatamente igual ao trecho do artigo.'}


In [ ]:
!cp /content/LLM6_C_reab_score.csv /content/drive/MyDrive/LLM6_C_reab_score.csv

# LLM 7 - Avaliador do Contexto x Artido Citado x Resposta

---



In [ ]:
# já avaliei se a relaçao artigo citado e contexto é veridica e se a resposta é veridica em relação a referencia
# quero saber se a resposta, o artido citado e o contexto são do mesmo tópico
class ANSWER_CONTEXT_RELATION_JSON(BaseModel):
    concordancias: str = Field(description="detalhe as concordancias entre resposta, artigo e contexto, se existir")
    discordancias: str = Field(description="detalhe as discordancias entre resposta, artigo e contexto, se existir")
    score: str = Field(description="score dado a coesão da resposta com os artigos e contexto")
    justificativa: str = Field(description="raciocínio usado para obter o score")

output_parser = JsonOutputParser(pydantic_object=ANSWER_CONTEXT_RELATION_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar.
Este TCC consiste em um FAQ a comunidade a entender melhor o Código Civil Brasileiro.
Sua tarefa então é avaliar por meio de um score a coerência entre a resposta, os artigos citados e o contexto fornecido.
Use os artigos de referência para entender a fonte de origem desses três elementos a serem avaliados.

Para o score use a seguinte fórmula:
score 0 - A resposta não foi retirada do artigos citados e a resposta não foi retirada do contexto fornecido.
score 1 - A resposta foi retirada do artigos citados, mas a resposta não foi retirada do contexto fornecido.
score 2 - A resposta não foi retirada do artigos citados, mas a resposta foi retirada do contexto fornecido.
score 3 - A resposta foi retirada do artigos citados e a resposta foi retirada do contexto fornecido, porém há discordâncias entre eles.
score 4 - A resposta foi retirada do artigos citados e a resposta foi retirada do contexto fornecido, e não há discordâncias entre eles.

Em sua justificativa descorra o raciocínio usado para obter o score.

Vamos começar a avaliação.
Artigos de Referênica (origem): {artigos_ref}
Resposta do FAQ: {resposta}
Artigos citados: {artigos_citados}
Contexto: {contexto}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos_ref", "resposta", "artigos_citados", "contexto"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
llm1_df = pd.read_csv("/content/LLM1_QA_generation_clean2.csv")
len(llm1_df)

277

In [ ]:
def call_AAC_coesion_score(llm1_sample):
    out = chain.invoke(
        {"artigos_ref": llm1_sample['ref_artigos'],
         "resposta": llm1_sample['resposta'],
         "artigos_citados": llm1_sample['artigos'],
         "contexto": llm1_sample['contexto']})
    return out

# test
call_AAC_coesion_score(llm1_df.iloc[4])

{'concordancias': 'A resposta está coerente com o artigo 1.807 e o contexto.',
 'discordancias': 'Não há discordâncias entre a resposta, o artigo e o contexto.',
 'score': '4',
 'justificativa': 'A resposta está coerente com o artigo 1.807, que estabelece que se o herdeiro não se pronunciar sobre a herança, presume-se que aceitou. O contexto também é coerente, pois o artigo 1.807 estabelece o prazo para o herdeiro se pronunciar sobre a herança.'}

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        ascore_json = call_AAC_coesion_score(llm1_sample)
        output_list.append(ascore_json)
        print(ascore_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        output_list.append('ERROR')
        continue

llm7_df = pd.DataFrame(output_list)
llm7_df.to_csv("LLM7_AAC_coesion_score.csv", index=False)

  0%|          | 1/277 [00:01<07:56,  1.73s/it]

{'concordancias': 'A resposta apresentada é uma lista de pessoas que não podem ser tutores, de acordo com o Art. 1.735 do Código Civil Brasileiro.', 'discordancias': '', 'score': '4', 'justificativa': 'A resposta foi retirada diretamente do artigo citado e do contexto fornecido, e não há discordâncias entre eles.'}


  1%|          | 2/277 [00:03<06:53,  1.50s/it]

{'concordancias': 'A resposta coincide com o artigo citado e o contexto fornecido.', 'discordancias': 'Não há discordâncias entre a resposta, o artigo citado e o contexto fornecido.', 'score': '4', 'justificativa': 'A resposta foi retirada do artigo citado e do contexto fornecido, e não há discordâncias entre eles.'}


  1%|          | 3/277 [00:13<24:51,  5.44s/it]

...

100%|█████████▉| 276/277 [1:47:24<00:25, 25.76s/it]

{'concordancias': 'A resposta está em conformidade com o artigo 1.801 do Código Civil Brasileiro.', 'discordancias': 'Não há discordâncias entre a resposta, o artigo citado e o contexto fornecido.', 'score': '4', 'justificativa': 'A resposta foi retirada do artigo 1.801 do Código Civil Brasileiro e está em conformidade com o contexto fornecido.'}


100%|██████████| 277/277 [1:47:51<00:00, 23.36s/it]

{'concordancias': 'A resposta foi retirada do artigo citado (Art. 1.800, § 4º) e do contexto fornecido.', 'discordancias': '', 'score': '4', 'justificativa': 'A resposta é uma transcrição literal do § 4º do Art. 1.800 do Código Civil Brasileiro, que trata da sucessão testamentária.'}


AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
for idx, out in enumerate(output_list):
    if out is None:
        ascore_json = call_AAC_coesion_score(llm1_df.iloc[idx])
        print(ascore_json)
        print(idx)
        print()
        output_list[idx] = ascore_json

{'concordancias': 'A resposta foi retirada do artigo citado (Art. 1.643) e está de acordo com o contexto fornecido.', 'discordancias': 'Não há discordâncias entre a resposta, o artigo citado e o contexto fornecido.', 'score': '4', 'justificativa': 'A resposta está correta e foi retirada do artigo 1.643, que permite aos cônjuges comprar coisas necessárias à economia doméstica sem autorização um do outro. Além disso, a resposta está de acordo com o contexto fornecido.'}
86

{'concordancias': 'A resposta coincide com o artigo 1.658 e o contexto, pois ambos mencionam que no regime de comunhão parcial, os bens adquiridos durante o casamento são comunicados, com exceções.', 'discordancias': 'Não há discordâncias entre a resposta, o artigo e o contexto.', 'score': '4', 'justificativa': 'A resposta foi retirada do artigo 1.658 e do contexto, e não há discordâncias entre eles. A resposta é coerente e precisa, pois menciona que os bens adquiridos durante o casamento são comunicados, com exceções

In [ ]:
llm7_df = pd.DataFrame(output_list)
llm7_df.to_csv("LLM7_AAC_coesion_score.csv", index=False)

In [ ]:
!cp /content/LLM7_AAC_coesion_score.csv /content/drive/MyDrive/LLM7_AAC_coesion_score.csv

In [ ]:
llm7_df['score'].value_counts()

score
4    269
2      4
0      2
1      2
Name: count, dtype: int64

# LLM 8 - Avaliador de pergunta, pouco contexto

In [ ]:
class QUESTION_COMPLETENESS_JSON(BaseModel):
    score: str = Field(description="score de completude dado a pergunta")
    justificativa: str = Field(description="razão pelo qual a pergunta recebeu esse score")
    pergunta_original: str = Field(description="pergunta avaliada")
    resposta_original: str = Field(description="resposta avaliada")
    pergunta_mais_completa: str = Field(description="pergunta mais completa que a pergunta avaliada")

output_parser = JsonOutputParser(pydantic_object=QUESTION_COMPLETENESS_JSON)

In [ ]:
prompt = """
Você é um brilhante professor de direito do Código Civil Brasileiro especialista nos livros IV e V: Direito da Família e Sucessões.
Você recebeu um trabalho de conclusão de curso (TCC) para avaliar.
Este TCC consiste em um FAQ a comunidade a entender melhor o Código Civil Brasileiro.
Sua tarefa então é avaliar por meio de um score o quão contextualizada está a pergunta.
Use os artigos e a resposta para entender melhor o contexto em que a pergunta se encontra.
AVALIE SOMENTE A AUSÊNCIA DE CONTEXTO NA PERGUNTA.

Para o score use a seguinte fórmula:
score 0 - A pergunta está muito enxuta e não é possível respondê-la sem o conhecimento prévio dos artigos
score 1 - A pergunta está razoável e pode ser respondida, porém um leigo não saberia dizer de que tópico do direito ela se trata.
score 2 - A pergunta está perfeitamente formulada e contextualizada.

Entenda por contexto o assunto do direito ou da vida das pessoas ao qual a pergunta se refere.
Para ajudar o aluno no TCC, além da avaliação adicione a pergunta do FAQ o contexto faltante.
A resposta original deve responder completamente a pergunta contextualizada, por isso:
MANTENHA a pergunta original!
NÃO invente na adição do contexto!
NUNCA cite o número do artigo!
NUNCA use sentenças como: "de acordo com as disposições do Código Civil Brasileiro" ou "de acordo com o Código Civil", etc.

Vamos começar a avaliação:
Artigos: {artigos}
Pergunta do FAQ: {pergunta}
Resposta do FAQ: {resposta}

Instruções de saída: {format_instructions}
"""

prompt_template = PromptTemplate(
    input_variables=["artigos", "pergunta", "resposta"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_question_completeness_score(llm1_sample):
    out = chain.invoke(
        {"artigos": llm1_sample['ref_artigos'],
         "pergunta": llm1_sample['pergunta'],
         "resposta": llm1_sample['resposta']})
    return out

# test
call_question_completeness_score(llm1_df.iloc[22])

{'score': '1',
 'justificativa': 'A pergunta está razoável e pode ser respondida, porém um leigo não saberia dizer de que tópico do direito ela se trata.',
 'pergunta_original': 'O que é a legítima?',
 'resposta_original': 'A legítima é a metade dos bens da herança que pertence aos herdeiros necessários.',
 'pergunta_mais_completa': 'No contexto da sucessão, o que é a legítima e quem são os herdeiros que têm direito a ela?'}

## Loop

In [ ]:
output_list = []
for i, llm1_sample in tqdm(llm1_df.iterrows(), total=llm1_df.shape[0]):
    try:
        score_json = call_question_completeness_score(llm1_sample)
        output_list.append(score_json)
        print(score_json)
    except Exception as e:
        print(f"Erro na iteração {i}: {e}")
        continue

llm8_df = pd.DataFrame(output_list)
llm8_df.to_csv("LLM8_Q_comp_score.csv", index=False)

  0%|          | 1/277 [00:01<05:31,  1.20s/it]

{'score': '1', 'justificativa': 'A pergunta está razoável e pode ser respondida, porém um leigo não saberia dizer de que tópico do direito ela se trata.', 'pergunta_original': 'Quem não pode ser tutor?', 'resposta_original': 'Aqueles que não tiverem a livre administração de seus bens, os inimigos do menor, os condenados por crime, as pessoas de mau procedimento, entre outros.', 'pergunta_mais_completa': 'Quem não pode ser tutor de um menor, de acordo com as disposições legais?'}


  1%|          | 2/277 [00:02<04:41,  1.02s/it]

{'score': '2', 'justificativa': 'A pergunta está perfeitamente formulada e contextualizada.', 'pergunta_original': 'O que acontece quando o herdeiro aceita a herança?', 'resposta_original': 'A transmissão da herança torna-se definitiva.', 'pergunta_mais_completa': 'O que acontece quando o herdeiro aceita a herança em relação à transmissão da propriedade?'}


  1%|          | 3/277 [00:19<38:15,  8.38s/it]

...

{'score': '2', 'justificativa': 'A pergunta está perfeitamente formulada e contextualizada.', 'pergunta_original': 'Quais são os legitimados a suceder?', 'resposta_original': 'As pessoas nascidas ou já concebidas no momento da abertura da sucessão, os filhos, ainda não concebidos, de pessoas indicadas pelo testador, desde que vivas estas ao abrir-se a sucessão, as pessoas jurídicas e as pessoas jurídicas, cuja organização for determinada pelo testador sob a forma de fundação.', 'pergunta_mais_completa': 'Quais são os legitimados a suceder em uma sucessão, de acordo com as disposições do Código Civil Brasileiro?'}


100%|█████████▉| 276/277 [1:48:18<00:24, 24.44s/it]

{'score': '2', 'justificativa': 'A pergunta está perfeitamente formulada e contextualizada.', 'pergunta_original': 'Quem não pode ser nomeado herdeiro ou legatário?', 'resposta_original': 'A pessoa que, a rogo, escreveu o testamento, nem o seu cônjuge ou companheiro, ou os seus ascendentes e irmãos, as testemunhas do testamento, o concubino do testador casado, salvo se este, sem culpa sua, estiver separado de fato do cônjuge há mais de cinco anos, e o tabelião, civil ou militar, ou o comandante ou escrivão, perante quem se fizer, assim como o que fizer ou aprovar o testamento.', 'pergunta_mais_completa': 'Quem não pode ser nomeado herdeiro ou legatário de acordo com as disposições do direito das sucessões?'}


100%|██████████| 277/277 [1:48:44<00:00, 23.55s/it]

{'score': '2', 'justificativa': 'A pergunta está perfeitamente formulada e contextualizada.', 'pergunta_original': 'O que acontece se o herdeiro esperado não for concebido?', 'resposta_original': 'Se, decorridos dois anos após a abertura da sucessão, não for concebido o herdeiro esperado, os bens reservados, salvo disposição em contrário do testador, caberão aos herdeiros legítimos.', 'pergunta_mais_completa': 'O que acontece se o herdeiro esperado, mencionado no testamento, não for concebido dentro do prazo de dois anos após a abertura da sucessão?'}


In [ ]:
score_json = call_question_completeness_score(llm1_df.iloc[114])
score_json

{'score': '1',
 'justificativa': 'A pergunta está razoável e pode ser respondida, porém um leigo não saberia dizer de que tópico do direito ela se trata.',
 'pergunta_original': 'Quais são os casos de indignidade que excluem um herdeiro?',
 'resposta_original': 'Os casos de indignidade que excluem um herdeiro são: homicídio doloso, acusação caluniosa em juízo e violência ou meios fraudulentos para inibir o autor da herança de dispor livremente de seus bens.',
 'pergunta_mais_completa': 'Quais são os casos de indignidade que excluem um herdeiro da sucessão, de acordo com as disposições do Código Civil Brasileiro?'}

In [ ]:
llm8_df = pd.DataFrame(output_list)
llm8_df.to_csv("LLM8_Q_comp_score.csv", index=False)

In [ ]:
llm8_df

,artigos_citados,contexto_existe_no_artigo,contexto,score,justificativa
0,Art. 1.735,Sim,Art. 1.735. Não podem ser tutores e serão exon...,3,O contexto apresentado pelo aluno é quase idên...
1,Art. 1.804,Sim,"Art. 1.804. Aceita a herança, torna-se definit...",3,"O contexto é idêntico ao artigo original, com ..."
2,Art. 1.805,Sim,"Art. 1.805. A aceitação da herança, quando exp...",3,"O contexto é idêntico ao artigo original, com ..."
3,Art. 1.806,Sim,Art. 1.806. A renúncia da herança deve constar...,3,"O contexto é idêntico ao artigo original, com ..."
4,Art. 1.807,Sim,Art. 1.807. O interessado em que o herdeiro de...,3,"O contexto é idêntico ao artigo original, com ..."
...,...,...,...,...,...
272,Art. 1.523,Sim,Não devem casar: I - o viúvo ou a viúva que ti...,3,O contexto é idêntico ao inciso I do artigo 1....
273,Art. 1.524,Sim,As causas suspensivas da celebração do casamen...,3,"O contexto é idêntico ao artigo original, com ..."
274,Art. 1.798,Sim,Legitimam-se a suceder as pessoas nascidas ou ...,3,"O contexto é idêntico ao artigo 1.798, com as ..."
275,Art. 1.801,Sim,Não podem ser nomeados herdeiros nem legatários:,2,O contexto apresenta a mesma estrutura e começ...


In [ ]:
!cp LLM8_Q_comp_score.csv /content/drive/MyDrive/LLM8_Q_comp_score.csv

cp: cannot stat 'LLM8_Q_comp_score.csv': No such file or directory


# Dados escolhidos avaliar se tem repetidos